# 06 – Model Comparison

In this notebook, we compare different sequence models to classify penalty kick direction from pose keypoints.

Models included:
- LSTM (baseline)
- Bidirectional LSTM
- GRU
- Lightweight Transformer

Evaluation is based on accuracy, F1-score, and training time.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Masking, Bidirectional
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

# Supposé : X, y sont chargés depuis notebook 03
y_cat = to_categorical(y, num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

In [ ]:
def evaluate_model(model):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0, validation_split=0.1)
    y_pred = model.predict(X_test)
    y_true = np.argmax(y_test, axis=1)
    y_pred_class = np.argmax(y_pred, axis=1)
    f1 = f1_score(y_true, y_pred_class, average='macro')
    acc = accuracy_score(y_true, y_pred_class)
    return acc, f1

In [ ]:
# Tester différents modèles
results = {}

# 1. LSTM simple
m1 = Sequential([
    Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])),
    LSTM(64),
    Dropout(0.3),
    Dense(3, activation='softmax')
])
results['LSTM'] = evaluate_model(m1)

# 2. Bidirectional LSTM
m2 = Sequential([
    Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(3, activation='softmax')
])
results['BiLSTM'] = evaluate_model(m2)

# 3. GRU
m3 = Sequential([
    Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])),
    GRU(64),
    Dropout(0.3),
    Dense(3, activation='softmax')
])
results['GRU'] = evaluate_model(m3)

# 4. Lightweight Transformer
from tensorflow.keras.layers import Dense, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model

inputs = Input(shape=(X.shape[1], X.shape[2]))
x = Dense(64)(inputs)
x = LayerNormalization()(x)
x = MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.3)(x)
outputs = Dense(3, activation='softmax')(x)

transformer_model = Model(inputs, outputs)
results['Transformer'] = evaluate_model(transformer_model)

In [ ]:
# Comparaison des scores
for model_name, (acc, f1) in results.items():
    print(f"{model_name}: Accuracy={acc:.2%}, F1-score={f1:.2%}")

plt.bar(results.keys(), [r[0] for r in results.values()], label='Accuracy')
plt.bar(results.keys(), [r[1] for r in results.values()], alpha=0.6, label='F1-score')
plt.ylabel("Score")
plt.title("Model Comparison")
plt.legend()
plt.show()